In [142]:
import os
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION=os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_DEPLOYMENT=os.getenv("AZURE_OPENAI_DEPLOYMENT")

In [143]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service
# If you encounter an error here are some ways to troubleshoot:
# - you can exclude the problematic credential by using a parameter (ex. exclude_shared_token_cache_credential=True)
# - For your credential, grant Cognitive Services User and Cognitive Services OpenAI User roles - EVEN IF YOU HAVE OWNER/ADMIN RIGHTS.
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_version=AZURE_OPENAI_API_VERSION,
  azure_ad_token_provider=token_provider
)

In [144]:
IMAGE_PATH="../media/whiteboard-full-sample.png"

In [145]:
import base64

def executeGptVision(encoded_image, system_prompt_file=None, system_prompt=None):
    if system_prompt_file:
        with open(system_prompt_file, "r") as file:
            system_prompt = file.read()
    elif system_prompt is None:
        raise ValueError("Either filename or system_prompt must be provided.")

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_image}"
                    }
                }
            ]
        }
    ]

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=messages,
        temperature=0
    )

    return response.choices[0].message.content

In [146]:
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
noteType = executeGptVision(encoded_image, system_prompt_file="../az-function/prompts/detectNoteType.txt")
print (noteType)

WHITEBOARD


In [147]:
if noteType == "PAPER":
    ocrFile = "../az-function/prompts/ocrPaper.txt"
elif noteType == "WHITEBOARD":
    ocrFile = "../az-function/prompts/ocrWhiteboard.txt"
else:
    ocrFile = "../az-function/prompts/ocrImage.txt"

extractedText = executeGptVision(encoded_image, system_prompt_file=ocrFile)
print (extractedText)

```markdown
## App Innovation in the Era of AI

Every app will be reinvented with AI.  
New apps will be built that weren't possible.

### Today (from MS Build - BRK120)

- Increased efficiency in business processes
- AI influence to better customer engagement and business growth
- AI is accelerating the need to move to the cloud.

### AI Enhanced Migration and Modernization

- Visual Studio - Azure Migrate extension, "Replatform to Azure"
- GitHub Copilot skills for Azure Migrate

### Modern Apps --> Intelligent Apps

- Buy a Copilot (M365, etc) vs. Build a Copilot (Copilot Stack, Pro Code)
- Copilot Studio (Low Code)

### AI Observability and Evaluation

- Prompt flow
- AI apps / AI pipeline

### AI Integration

- AI Gateways
  - API Mgmt
  - AI Hub
- Logic Apps
- Power Automate

## App Dev & Design in the Era of AI

### Chatbot Overload: Every New App Will Have a Chatbot

Many existing apps are implementing a chatbot.

### Chatbot (and Intelligent) Search

- Design apps with this in

In [148]:
def executeCompletion(text, system_prompt_file=None, system_prompt=None):
    if system_prompt_file:
        with open(system_prompt_file, "r") as file:
            system_prompt = file.read()
    elif system_prompt is None:
        raise ValueError("Either filename or system_prompt must be provided.")

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": text
        }
    ]

    response = client.chat.completions.create(
        model="gpt4o",
        messages=messages,
        temperature=0.3
    )

    return response.choices[0].message.content

In [149]:
if noteType == "PAPER" or noteType == "WHITEBOARD":
    extractedText = executeCompletion(text=extractedText, system_prompt_file="../az-function/prompts/proofread.txt")
    print (extractedText)

```markdown
## App Innovation in the Era of AI

Every app will be reinvented with AI.  
New apps will be built that weren't possible.

### Today (from Microsoft Build - BRK120)

- Increased efficiency in business processes
- AI influence to better customer engagement and business growth
- AI is accelerating the need to move to the cloud.

### AI Enhanced Migration and Modernization

- Visual Studio - Azure Migrate extension, "Replatform to Azure"
- GitHub Copilot skills for Azure Migrate

### Modern Apps --> Intelligent Apps

- Buy a Copilot (Microsoft 365, etc.) vs. Build a Copilot (Copilot Stack, Pro Code)
- Copilot Studio (Low Code)

### AI Observability and Evaluation

- Prompt flow
- AI apps / AI pipeline

### AI Integration

- AI Gateways
  - API Management
  - AI Hub
- Logic Apps
- Power Automate

## App Development & Design in the Era of AI

### Chatbot Overload: Every New App Will Have a Chatbot

Many existing apps are implementing a chatbot.

### Chatbot (and Intelligent) Sea

In [150]:
if noteType == "PAPER" or noteType == "WHITEBOARD":
    extractedText = executeCompletion(extractedText, system_prompt_file="../az-function/prompts/sectionHeader.txt")
    print (extractedText)

```markdown
## App Innovation in the Era of AI

Every app will be reinvented with AI.  
New apps will be built that weren't possible.

### Current Trends

- Increased efficiency in business processes
- AI influence to better customer engagement and business growth
- AI is accelerating the need to move to the cloud.

### Migration Tools

- Visual Studio - Azure Migrate extension, "Replatform to Azure"
- GitHub Copilot skills for Azure Migrate

### Intelligent Apps

- Buy a Copilot (Microsoft 365, etc.) vs. Build a Copilot (Copilot Stack, Pro Code)
- Copilot Studio (Low Code)

### Observability & Evaluation

- Prompt flow
- AI apps / AI pipeline

### Integration Tools

- AI Gateways
  - API Management
  - AI Hub
- Logic Apps
- Power Automate

## App Development & Design in the Era of AI

### Chatbot Proliferation

Many existing apps are implementing a chatbot.

### Intelligent Search

- Design apps with this in mind
  - e.g., remove extensive and deep website navigation menus
- Make onli

In [151]:
extractedText = extractedText.strip()
if extractedText.startswith("```markdown"):
    extractedText = extractedText[len("```markdown"):]
if extractedText.endswith("```"):
    extractedText = extractedText[:-len("```")]

extractedText = extractedText.strip()

In [152]:
import os
import datetime

filename = os.path.basename(IMAGE_PATH)
last_updated = datetime.datetime.fromtimestamp(os.path.getmtime(IMAGE_PATH)).strftime('%Y-%m-%d %H:%M')

result = f"""---
note-type: {noteType}
created-date: {last_updated}
last-updated: {last_updated}
---
# {filename}
![{filename}]({IMAGE_PATH})
{extractedText}"""

# save result to output.md
with open("output.md", "w") as file:
    file.write(result)

print(result)


---
note-type: WHITEBOARD
created-date: 2024-07-02 18:34
last-updated: 2024-07-02 18:34
---
# 20240704 App Inno in the Era of AI.png
![20240704 App Inno in the Era of AI.png](../media/20240704 App Inno in the Era of AI.png)
## App Innovation in the Era of AI

Every app will be reinvented with AI.  
New apps will be built that weren't possible.

### Current Trends

- Increased efficiency in business processes
- AI influence to better customer engagement and business growth
- AI is accelerating the need to move to the cloud.

### Migration Tools

- Visual Studio - Azure Migrate extension, "Replatform to Azure"
- GitHub Copilot skills for Azure Migrate

### Intelligent Apps

- Buy a Copilot (Microsoft 365, etc.) vs. Build a Copilot (Copilot Stack, Pro Code)
- Copilot Studio (Low Code)

### Observability & Evaluation

- Prompt flow
- AI apps / AI pipeline

### Integration Tools

- AI Gateways
  - API Management
  - AI Hub
- Logic Apps
- Power Automate

## App Development & Design in the Er